In [17]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [18]:
from rake_nltk import Rake
import pandas as pd
import requests
from IPython.display import Image, display
from timeit import default_timer as timer
import pprint
import sys
from timeit import default_timer as timer
import gensim
import string

In [19]:
# Utilities
def print_song_keywords(Song_Name):
    rake = Rake()
    rake.extract_keywords_from_text(get_song(Song_Name))
    for i in range(0,len(rake.get_ranked_phrases_with_scores())):
        print(str(rake.get_ranked_phrases_with_scores()[i]))

def extract_song_keywords(Song_Name):
    rake = Rake()
    return rake.extract_keywords_from_text(get_song(Song_Name))

def get_song(title):
    return Songs.loc[title]["text"]

def print_song(title):
    print(get_song(title))
    
def createDict(keywords,N):
    Dict = {}
    r = requests.get('https://pixabay.com/api/?key=6713313-c85a6d8e3f6fe1a13b85f7d78&q='+"+".join(keywords)+'&image_type=photo&page=1&per_page='+str(N))
    for img in r.json()['hits']:
        Dict[img['webformatURL']] = img["tags"].split(",")
    return Dict

def displayDict(Dict,limit):
    i = 1
    urls = Dict.keys()
    
    for url in urls:
        print(url)
        print(Dict[url])
        display(Image(url,height=700,width=1000,embed=True))
        print("\n")
        if(i == limit): 
            break
        i += 1

In [20]:
breaker = "\n \n"
# Pre-processing
Songs = pd.read_csv("songdata.csv")
Songs.set_index("song",inplace=True)
Songs.drop(["artist","link"],inplace=True,axis=1)

In [21]:
rake = Rake()
rake.extract_keywords_from_text('Almost heaven, West Virginia,  Blue ridge mountain, Shenandoah river,  Life is old there, older than the trees,  Younger than the mountains, growing like a breeze ')
#rake.get_ranked_phrases_with_scores()

In [22]:
# Song Name -> List of URLS

Max_number_of_search_results = 200

start    = timer()
keywords = ['radio']
Dict     = createDict(keywords, Max_number_of_search_results)

print("\nTime taken:           " +  str(timer() - start) + "\nSize of Table Element:         " + str(len(Dict.keys())))


Time taken:           0.8292016652468419
Size of Table Element:         200


In [23]:
start = timer()
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',limit=1000000,binary=True)
print(str(timer() - start))

18.928638819389562


In [24]:
nlp = model.wv

In [28]:
tags = list(Dict.values())[4]


tags

['microphone', ' recording', ' studio']

In [26]:
# vector of similiar words to each tag
for tag in tags:
    # just a way to convert keywords into a form that can be submitted to a query
    filteredform = "".join((char for char in tag.lower().strip() if char not in string.punctuation))
    print(tag + "-> " + str(model.wv.most_similar(filteredform,topn=10)))
    print("\n")


microphone-> [('microphones', 0.7724970579147339), ('mic', 0.7113080024719238), ('mike', 0.6754544377326965), ('mikes', 0.6500441431999207), ('headset_microphone', 0.6241725087165833), ('mics', 0.6153472661972046), ('tape_recorder', 0.6092134118080139), ('lapel_microphone', 0.600554883480072), ('earpiece', 0.5868943929672241), ('lectern', 0.5765868425369263)]


 recording-> [('Recording', 0.6919281482696533), ('recordings', 0.6843935251235962), ('recorded', 0.667299211025238), ('Recorded', 0.5864580869674683), ('audio', 0.5294467210769653), ('rerecording', 0.5183216333389282), ('mixdown', 0.5135234594345093), ('CD', 0.5127924680709839), ('overdub', 0.5019869208335876), ('recording_studio', 0.500024676322937)]


 studio-> [('studios', 0.7762775421142578), ('Studios', 0.6829925775527954), ('recording_studio', 0.6573970913887024), ('Studio', 0.6525226831436157), ('postproduction', 0.5414010286331177), ('Recording_Studios', 0.5307695865631104), ('soundstage', 0.5282276272773743), ('Warner_

In [27]:
s = "Does this string. Have punctuation?"

"".join((char for char in s.lower().strip() if char not in string.punctuation))

'does this string have punctuation'